In [47]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 70 kB/s 


In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline

In [49]:
df = pd.read_csv('/content/drive/MyDrive/f1/qualifying.csv', index_col=0)

In [46]:
df

,season,round,driver_id,temp,rain,clear,overcast,partially_cloudy,precipitation,wind,...,rodriguez,sepang,shanghai,silverstone,sochi,spa,suzuka,villeneuve,yas_marina,zandvoort
0,2014,1,4.0,21.5,1.0,0.0,0.0,1.0,0.0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,2,4.0,27.4,1.0,0.0,0.0,1.0,0.0,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,4,4.0,14.4,1.0,0.0,1.0,0.0,0.0,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,5,4.0,20.8,0.0,1.0,0.0,0.0,0.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,2014,13,4.0,20.5,1.0,0.0,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2022,9,18.0,11.6,1.0,0.0,0.0,0.0,1.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
169,2022,10,14.0,14.0,1.0,0.0,0.0,1.0,0.0,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
170,2022,11,18.0,15.4,1.0,0.0,0.0,0.0,1.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,2022,12,7.0,27.5,0.0,1.0,0.0,0.0,0.0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df.shape

(144, 41)

In [50]:
y = df['driver_id']
X = df.drop(['driver_id'], axis=1)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [52]:
from sklearn import preprocessing
st_scaler = preprocessing.StandardScaler()
X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

In [ ]:
params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'lbfgs'],
        'C': np.logspace(-3,1,20)}

logreg = LogisticRegression(class_weight='balanced', max_iter=10000)
logreg_cv = GridSearchCV(logreg, params, scoring='f1_micro')
logreg_cv.fit(X_train, y_train)
logreg_predictions = logreg_cv.best_estimator_.predict(X_test)

In [ ]:
logreg_cv.best_score_

0.5626315789473685

In [ ]:
f1_score(y_test, logreg_cv.best_estimator_.predict(X_test), average='micro')

0.14285714285714285

In [ ]:
params={'criterion': ['gini', 'entropy'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}
        
rfc = RandomForestClassifier(class_weight='balanced_subsample')
rfc_cv = GridSearchCV(rfc, params, scoring='f1_micro')
rfc_cv.fit(X_train, y_train)
rfc_predictions = rfc_cv.best_estimator_.predict(X_test)

In [ ]:
rfc_cv.best_score_

0.5526315789473685

In [ ]:
f1_score(y_test, rfc_cv.best_estimator_.predict(X_test), average='micro')

0.4523809523809524

In [ ]:
params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'decision_function_shape' : ['ovo', 'ovr']} 

svc = SVC(class_weight='balanced')
svc_cv = GridSearchCV(svc, params, scoring='f1_micro')
svc_cv.fit(X_train, y_train)
svc_predictions = svc_cv.best_estimator_.predict(X_test)

In [ ]:
svc_cv.best_score_

0.4889473684210527

In [ ]:
f1_score(y_test, svc_cv.best_estimator_.predict(X_test), average='micro')

0.47619047619047616

In [53]:
params = {'depth': [4,5,6,7,8,9, 10],
          'learning_rate' : [0.01,0.02,0.03,0.04],
          'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]}

cbc = CatBoostClassifier(loss_function='MultiClass')
cbc_cv = GridSearchCV(cbc, params, scoring='f1_micro')
cbc_cv.fit(X_train, y_train)
cbc_cv_predictions = cbc_cv.best_estimator_.predict(X_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
20:	learn: 1.9835027	total: 296ms	remaining: 831ms
21:	learn: 1.9766692	total: 317ms	remaining: 836ms
22:	learn: 1.9669693	total: 328ms	remaining: 813ms
23:	learn: 1.9604556	total: 339ms	remaining: 792ms
24:	learn: 1.9527014	total: 372ms	remaining: 819ms
25:	learn: 1.9461589	total: 400ms	remaining: 831ms
26:	learn: 1.9353115	total: 402ms	remaining: 790ms
27:	learn: 1.9282792	total: 427ms	remaining: 792ms
28:	learn: 1.9218816	total: 448ms	remaining: 787ms
29:	learn: 1.9146046	total: 454ms	remaining: 757ms
30:	learn: 1.9087365	total: 476ms	remaining: 752ms
31:	learn: 1.9024623	total: 487ms	remaining: 730ms
32:	learn: 1.8944720	total: 509ms	remaining: 725ms
33:	learn: 1.8873141	total: 512ms	remaining: 692ms
34:	learn: 1.8815647	total: 533ms	remaining: 685ms
35:	learn: 1.8733917	total: 539ms	remaining: 659ms
36:	learn: 1.8659049	total: 560ms	remaining: 651ms
37:	learn: 1.8534327	total: 582ms	remaining: 644ms
38:	learn: 1.8

In [54]:
cbc_cv.best_score_

0.5599999999999999

In [55]:
f1_score(y_test, cbc_cv.best_estimator_.predict(X_test), average='micro')

0.6818181818181818

In [56]:
cbc_cv.best_params_


{'depth': 10, 'iterations': 100, 'learning_rate': 0.02}

In [ ]:
params = {
    'max_depth': [2,3,4,5,6,7,8,9,10],
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05],
    'scale_pos_weight' : [1, 10, 25, 50, 75, 99, 100, 1000]}

sample_weights = compute_sample_weight(class_weight='balanced', y=y)

xgb = XGBClassifier(n_jobs = -1, objective='multi:softmax', sample_weight=sample_weights)
xgb_cv = GridSearchCV(xgb, params, scoring='f1_micro')
xgb_cv.fit(X_train, y_train)
xgb_predictions = xgb_cv.best_estimator_.predict(X_test)

In [ ]:
xgb_cv.best_score_

0.5731578947368421

In [ ]:
f1_score(y_test, xgb_cv.best_estimator_.predict(X_test), average='micro')

0.5476190476190477

In [ ]:
xgb_cv.best_params_


{'learning_rate': 0.01,
 'max_depth': 2,
 'n_estimators': 100,
 'scale_pos_weight': 1}

In [63]:
q_pipe = Pipeline([('st_scaler', preprocessing.StandardScaler()), 
                   ('cbc_cv', CatBoostClassifier(loss_function='MultiClass',
                    depth= 10, iterations= 100, learning_rate= 0.02))])

In [64]:
q_pipe.fit(X,y)

0:	learn: 2.3907395	total: 44.1ms	remaining: 4.36s
1:	learn: 2.3701094	total: 45.7ms	remaining: 2.24s
2:	learn: 2.3525275	total: 79.3ms	remaining: 2.56s
3:	learn: 2.3359870	total: 96.7ms	remaining: 2.32s
4:	learn: 2.3233111	total: 131ms	remaining: 2.49s
5:	learn: 2.3073539	total: 149ms	remaining: 2.33s
6:	learn: 2.2953117	total: 168ms	remaining: 2.23s
7:	learn: 2.2852522	total: 203ms	remaining: 2.33s
8:	learn: 2.2767403	total: 238ms	remaining: 2.4s
9:	learn: 2.2680261	total: 277ms	remaining: 2.49s
10:	learn: 2.2575821	total: 295ms	remaining: 2.39s
11:	learn: 2.2481867	total: 300ms	remaining: 2.2s
12:	learn: 2.2355291	total: 333ms	remaining: 2.23s
13:	learn: 2.2246043	total: 338ms	remaining: 2.07s
14:	learn: 2.2150914	total: 374ms	remaining: 2.12s
15:	learn: 2.2033652	total: 408ms	remaining: 2.14s
16:	learn: 2.1934710	total: 442ms	remaining: 2.16s
17:	learn: 2.1862772	total: 486ms	remaining: 2.21s
18:	learn: 2.1683499	total: 520ms	remaining: 2.22s
19:	learn: 2.1584793	total: 554ms	remai

Pipeline(steps=[('st_scaler', StandardScaler()),
                ('cbc_cv',
                 <catboost.core.CatBoostClassifier object at 0x7fba1d1d5ed0>)])

In [65]:
pickle.dump(q_pipe, open('cb_model.sav', 'wb'))

In [ ]:
pickle.dump(list(X.columns), open('X_q.sav', 'wb'))